In [3]:
import csv
import torch
from tqdm import tqdm
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel

template_file = "/home/vbertalan/Downloads/gpt2_logs_mini.log_templates.csv"
log_file = "/home/vbertalan/Downloads/gpt2_logs_mini.log"

# Auxiliary function to read lines from a raw log file
def read_lines_from_file(filename):
    with open(filename, "r", encoding="utf-8") as f:
        return [line.strip() for line in f.readlines() if line.strip()]

# Auxiliary function to extract templates from previous CSV file
def extract_templates(csv_file):
    event_templates = []
    with open(csv_file, mode='r', encoding='utf-8') as arquivo:
        leitor = csv.DictReader(arquivo)
        for linha in leitor:
            if 'EventTemplate' in linha:
                event_templates.append(linha['EventTemplate'])
    return event_templates


# Step 1: Load GPT-2 tokenizer and add custom log templates
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
log_templates =  extract_templates(template_file)

# Add log templates as tokens to the tokenizer
tokenizer.add_tokens(log_templates)

# Set padding token to EOS
tokenizer.pad_token = tokenizer.eos_token

# Step 2: Prepare log sequences (you may add your actual log data here)
sequences = read_lines_from_file(log_file)

# Tokenize sequences
tokenized_sequences = tokenizer(
    sequences,  # List of sentences
    truncation=True,
    padding=True,  # Padding all sequences to the same length
    max_length=128,  # Set max length for input sequences
    return_tensors="pt"  # Return PyTorch tensors
)

# Step 3: Load pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Resize the model's embeddings to include the new tokens
model.resize_token_embeddings(len(tokenizer))

# Step 4: Prepare Dataset and DataLoader for training
class LogSequenceDataset(Dataset):
    def __init__(self, tokenized_sequences):
        self.input_ids = tokenized_sequences['input_ids']
        self.attention_mask = tokenized_sequences['attention_mask']
        
    def __len__(self):
        return self.input_ids.shape[0]
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx]
        }

# Create the dataset and dataloader
dataset = LogSequenceDataset(tokenized_sequences)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Step 5: Set up optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Step 6: Enable gradient updates on embeddings
model.get_input_embeddings().requires_grad_(True)

# Step 7: Train the model (Continual Pretraining)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 3  # Adjust based on your training needs

for epoch in range(epochs):
    model.train()
    loop = tqdm(dataloader, leave=True)
    
    for batch in loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        
        optimizer.zero_grad()
        
        # Forward pass (language modeling task)
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        
        # Backward pass
        loss.backward()
        
        # Update the model's parameters
        optimizer.step()
        
        loop.set_postfix(loss=loss.item())

    print(f"Epoch {epoch + 1}/{epochs} completed.")

# Save the fine-tuned model
model.save_pretrained("fine_tuned_gpt2")
tokenizer.save_pretrained("fine_tuned_gpt2")

print("Fine-tuning completed and model saved.")


100%|██████████| 10/10 [00:06<00:00,  1.65it/s, loss=nan]


Epoch 1/3 completed.


100%|██████████| 10/10 [00:05<00:00,  1.73it/s, loss=nan]


Epoch 2/3 completed.


100%|██████████| 10/10 [00:05<00:00,  1.73it/s, loss=nan]


Epoch 3/3 completed.
Fine-tuning completed and model saved.


In [ ]:
import torch
from tqdm import tqdm
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login

# === Step 0: Login on Hugging Face (coloque seu token aqui se necessário) ===
# Step 0: Code for reading HuggingFace token
def get_huggingface_token():
    f = open("huggingface_token.txt", "r")
    return (f.read())

login(token=get_huggingface_token())  # Descomente se preferir login via código

# === Step 1: Define LLaMA 2 model ===
model_name = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
tokenizer.pad_token = tokenizer.eos_token

# === Step 2: Add custom tokens ===
log_templates = [
    "Error encountered in module X",
    "Error encountered in",
    "Unexpected behavior in network communication",
    "System rebooted successfully",
    "Segmentation fault in memory allocation"
]
tokenizer.add_tokens(log_templates)

# === Step 3: Prepare sequences ===
sequences = [
    "Error encountered in module X The weather is great today. I am working hard.",
    "The system rebooted successfully after the error."
]

tokenized_sequences = tokenizer(
    sequences,
    truncation=True,
    padding=True,
    max_length=128,
    return_tensors="pt"
)

# === Step 4: Load LLaMA 2 model ===
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    use_auth_token=True,
    torch_dtype=torch.float16,
    device_map="auto"
)

# === Step 5: Resize embeddings to include new tokens ===
model.resize_token_embeddings(len(tokenizer))

# === Step 6: Prepare Dataset & Dataloader ===
class LogSequenceDataset(Dataset):
    def __init__(self, tokenized_sequences):
        self.input_ids = tokenized_sequences['input_ids']
        self.attention_mask = tokenized_sequences['attention_mask']
        
    def __len__(self):
        return self.input_ids.shape[0]
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx]
        }

dataset = LogSequenceDataset(tokenized_sequences)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# === Step 7: Optimizer ===
optimizer = AdamW(model.parameters(), lr=5e-5)

# === Step 8: Train the model ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

epochs = 3
for epoch in range(epochs):
    model.train()
    loop = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}", leave=True)
    
    for batch in loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        loop.set_postfix(loss=loss.item())

# === Step 9: Save fine-tuned model ===
model.save_pretrained("fine_tuned_llama2")
tokenizer.save_pretrained("fine_tuned_llama2")
print("✅ Fine-tuning completed and model saved.")


tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

: 

In [ ]:
import torch
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from tqdm import tqdm

template_file = ""
log_file = "gpt2_logs.log"

# Função auxiliar para ler linhas de um arquivo como lista
def read_lines_from_file(filename):
    with open(filename, "r", encoding="utf-8") as f:
        return [line.strip() for line in f.readlines() if line.strip()]

# Step 1: Load GPT-2 tokenizer and add custom log templates
tokenizer = GPT2Tokenizer.from_pretrained("llama2-7b")

# Carregar log templates de arquivo externo
log_templates = read_lines_from_file(template_file)

# Adicionar os log templates como novos tokens no tokenizer
tokenizer.add_tokens(log_templates)

# Definir token de padding
tokenizer.pad_token = tokenizer.eos_token

# Step 2: Carregar sequências de log de arquivo externo
sequences = read_lines_from_file(log_file)

# Tokenizar as sequências
tokenized_sequences = tokenizer(
    sequences,
    truncation=True,
    padding=True,
    max_length=128,
    return_tensors="pt"
)

# Step 3: Load pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

# Step 4: Dataset e DataLoader
class LogSequenceDataset(Dataset):
    def __init__(self, tokenized_sequences):
        self.input_ids = tokenized_sequences['input_ids']
        self.attention_mask = tokenized_sequences['attention_mask']
        
    def __len__(self):
        return self.input_ids.shape[0]
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx]
        }

dataset = LogSequenceDataset(tokenized_sequences)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Step 5: Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Step 6: Enable embedding updates
model.get_input_embeddings().requires_grad_(True)

# Step 7: Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 3

for epoch in range(epochs):
    model.train()
    loop = tqdm(dataloader, leave=True)
    
    for batch in loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
        loop.set_postfix(loss=loss.item())

    print(f"Epoch {epoch + 1}/{epochs} completed.")

# Salvar modelo e tokenizer
model.save_pretrained("fine_tuned_gpt2")
tokenizer.save_pretrained("fine_tuned_gpt2")

print("Fine-tuning completed and model saved.")

/home/vbertalan/anaconda3/lib/python3.9/site-packages/torch/amp/grad_scaler.py:131: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
Epoch 1/3:   0%|          | 0/1 [00:00<?, ?it/s]/home/vbertalan/anaconda3/lib/python3.9/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Epoch 3/3: 100%|██████████| 1/1 [00:00<00:00,  1.69it/s, loss=61.3]


Fine-tuning completed and model saved.


In [11]:
from transformers import GPT2Tokenizer

# === CONFIGURAÇÃO ===
LOG_FILE_PATH = "/home/vbertalan/Downloads/gpt2_logs.log"  # Caminho para seu arquivo .log

# === Lê o arquivo de logs ===
with open(LOG_FILE_PATH, "r", encoding="utf-8") as f:
    log_lines = [line.strip() for line in f if line.strip()]

# === Inicializa o tokenizer GPT-2 ===
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# === Calcula os comprimentos em tokens ===
token_lens = [len(tokenizer(log)["input_ids"]) for log in log_lines]

# === Exibe estatísticas ===
print(f"Total de logs: {len(token_lens)}")
print(f"Comprimento médio: {sum(token_lens)/len(token_lens):.2f} tokens")
print(f"Comprimento máximo: {max(token_lens)} tokens")


Token indices sequence length is longer than the specified maximum sequence length for this model (1026 > 1024). Running this sequence through the model will result in indexing errors


Total de logs: 652096
Comprimento médio: 156.45 tokens
Comprimento máximo: 1831 tokens
